# FastAPI Product Search API Testing

This notebook tests all endpoints of the Product Search API.

**Prerequisites:**
- Make sure your FastAPI app is running: `python app.py`
- The app should be accessible at http://localhost:8001

In [3]:
pip install -r requirement.txt

  Obtaining dependency information for fastapi==0.104.1 from https://files.pythonhosted.org/packages/f3/4f/0ce34195b63240b6693086496c9bab4ef23999112184399a3e88854c7674/fastapi-0.104.1-py3-none-any.whl.metadata
  Using cached fastapi-0.104.1-py3-none-any.whl.metadata (24 kB)
  Obtaining dependency information for uvicorn[standard]==0.24.0 from https://files.pythonhosted.org/packages/ed/0c/a9b90a856bbdd75bf71a1dd191af1e9c9ac8a272ed337f7200950c3d3dd4/uvicorn-0.24.0-py3-none-any.whl.metadata
  Using cached uvicorn-0.24.0-py3-none-any.whl.metadata (6.4 kB)
  Obtaining dependency information for pydantic==2.5.0 from https://files.pythonhosted.org/packages/d7/10/ddfb9539a6e55f7dfd6c2b9b81d86fcba2761ba87eeb81f8b1012957dcdc/pydantic-2.5.0-py3-none-any.whl.metadata
  Using cached pydantic-2.5.0-py3-none-any.whl.metadata (174 kB)
  Obtaining dependency information for requests==2.31.0 from https://files.pythonhosted.org/packages/70/8e/0e2d847013cb52cd35b38c009bb167a1a26b2ce6cd6965bf26b47bc0bf44/r

In [5]:
pip install pandas

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/1b/cc/0af9c07f8d714ea563b12383a7e5bde9479cf32413ee2f346a9c5a801f22/pandas-2.3.0-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Using cached pandas-2.3.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (91 kB)
  Obtaining dependency information for numpy>=1.23.2 from https://files.pythonhosted.org/packages/29/6b/2d31da8e6d2ec99bed54c185337a87f8fbeccc1cd9804e38217e92f3f5e2/numpy-2.3.0-cp311-cp311-macosx_14_0_arm64.whl.metadata
  Using cached numpy-2.3.0-cp311-cp311-macosx_14_0_arm64.whl.metadata (62 kB)
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/81/c4/34e93fe5f5429d7570ec1fa436f1986fb1f00c3e0f43a589fe2bbcd22c3f/pytz-2025.2-py2.py3-none-any.whl.metadata
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Obtaining dependency information for tzdata>=2022.7 from https://files.pythonhosted.org/packages/5c/23/c7abc0ca0a1526a0774eca151daeb8de6

In [7]:
# Import required libraries
import requests
import json
import pandas as pd
from datetime import datetime

# API Configuration
BASE_URL = "http://localhost:8001"
HEADERS = {"Content-Type": "application/json"}

print(f"API Base URL: {BASE_URL}")
print(f"Test Time: {datetime.now()}")

API Base URL: http://localhost:8001
Test Time: 2025-06-15 23:11:59.349473


## 1. Health Check Test
Basic endpoint to verify the API is running

In [8]:
# Test Health Check
print("=== Testing Health Check ===\n")

try:
    response = requests.get(f"{BASE_URL}/health", timeout=10)
    
    print(f"Status Code: {response.status_code}")
    print(f"Response Time: {response.elapsed.total_seconds():.2f}s")
    
    if response.status_code == 200:
        result = response.json()
        print("✅ Health check passed!")
        print(f"Status: {result['status']}")
        print(f"Timestamp: {result['timestamp']}")
    else:
        print(f"❌ Health check failed: {response.text}")
        
except requests.exceptions.RequestException as e:
    print(f"❌ Connection error: {e}")
    print("Make sure your FastAPI app is running at http://localhost:8001")

=== Testing Health Check ===

Status Code: 200
Response Time: 0.02s
✅ Health check passed!
Status: healthy
Timestamp: 1750003922.469975


## 2. Finding Documents Test
Test the document/product search functionality

In [ ]:
# Test Finding Documents - Red Dress Query
print("=== Testing Finding Documents (Red Dress) ===\n")

payload = {
    "user_query": """"{
  "messages": [
    {
      "role": "user",
      "content": "Hey, do you have any good moisturizers for dry skin?"
    },
    {
      "role": "assistant",
      "content": "Yes, we have a few great options for dry skin. You might like our Hydrating Aloe Cream and the Deep Moisture Repair Balm. Both are designed for long-lasting hydration."
    },
    {
      "role": "user",
      "content": "Actually, I’ve been thinking of setting up a small home gym. Do you sell dumbbells or resistance bands?"
    },
    {
      "role": "assistant",
      "content": "We currently have resistance bands in stock, including light, medium, and heavy sets. Dumbbells are temporarily out of stock but should be back in next week."
    },
    {
      "role": "user",
      "content": "Can I have red ones, hemes brand?"
    }
  ]
}""",
    "image_prompt": "picture of a red band that looks like a resistant band called hemes"
}


# messages = [
#     {"role": "user", "content": f"Generate 1-3 search queries for: {request.user_query}. Image context: {request.image_prompt}. Return as JSON array of strings."}
# ]

try:
    response = requests.post(
        f"{BASE_URL}/finding_documents",
        json=payload,
        headers=HEADERS,
        timeout=30
    )
    
    print(f"Status Code: {response.status_code}")
    print(f"Response Time: {response.elapsed.total_seconds():.2f}s")
    
    if response.status_code == 200:
        result = response.json()
        print("✅ Finding documents successful!")
        print(f"\nFound {len(result['results'])} products:\n")
        
        for i, item in enumerate(result['results'], 1):
            print(f"📦 Product {i}:")
            print(f"   Brand: {item['brand_name']}")
            print(f"   Description: {item['description']}")
            print(f"   Materials: {item['materials']}")
            print(f"   Price: {item['price']}")
            print(f"   Location: {item['location']}")
            # print(f"   Thinking: {item['thinking']}")
            print()
    else:
        print(f"❌ Request failed: {response.text}")
        
except requests.exceptions.RequestException as e:
    print(f"❌ Request error|: {e}")

=== Testing Finding Documents (Red Dress) ===

Status Code: 200
Response Time: 1.30s
✅ Finding documents successful!

Found 1 products:

📦 Product 1:
   Brand: The Mall Fashion
   Description: Classic red summer dress perfect for casual occasions
   Materials: 100% Cotton
   Price: $49.99
   Location: Floor 2, Section A



In [ ]:
# Test Finding Documents - Blue Jeans Query
print("=== Testing Finding Documents (Blue Jeans) ===\n")

payload = {
    "user_query": "I need blue jeans for work",
    "image_prompt": "professional casual wear"
}

try:
    response = requests.post(
        f"{BASE_URL}/finding_documents",
        json=payload,
        headers=HEADERS,
        timeout=30
    )
    
    print(f"Status Code: {response.status_code}")
    print(f"Response Time: {response.elapsed.total_seconds():.2f}s")
    
    if response.status_code == 200:
        result = response.json()
        print("✅ Finding documents successful!")
        
        # Display as a table
        products_data = []
        for item in result['results']:
            products_data.append({
                'Brand': item['brand_name'],
                'Description': item['description'][:50] + '...' if len(item['description']) > 50 else item['description'],
                'Price': item['price'],
                'Location': item['location']
            })
        
        df = pd.DataFrame(products_data)
        print("\n📊 Products Table:")
        print(df.to_string(index=False))
    else:
        print(f"❌ Request failed: {response.text}")
        
except requests.exceptions.RequestException as e:
    print(f"❌ Request error: {e}")

## 3. Image Captioning Test
Test the image processing functionality

In [ ]:
# Test Image Captioning - Dress Image
print("=== Testing Image Captioning (Dress) ===\n")

payload = {
    "image_path": "images/red_dress_photo.jpg"
}

try:
    response = requests.post(
        f"{BASE_URL}/image_captioning",
        json=payload,
        headers=HEADERS,
        timeout=15
    )
    
    print(f"Status Code: {response.status_code}")
    print(f"Response Time: {response.elapsed.total_seconds():.2f}s")
    
    if response.status_code == 200:
        result = response.json()
        print("✅ Image captioning successful!")
        print(f"\n🖼️  Caption: {result['results']}")
        print(f"⏱️  Processing Time: {result['processing_time']:.4f}s")
    else:
        print(f"❌ Request failed: {response.text}")
        
except requests.exceptions.RequestException as e:
    print(f"❌ Request error: {e}")

In [ ]:
# Test Image Captioning - Shoes Image
print("=== Testing Image Captioning (Shoes) ===\n")

payload = {
    "image_path": "images/brown_shoes_display.jpg"
}

try:
    response = requests.post(
        f"{BASE_URL}/image_captioning",
        json=payload,
        headers=HEADERS,
        timeout=15
    )
    
    print(f"Status Code: {response.status_code}")
    print(f"Response Time: {response.elapsed.total_seconds():.2f}s")
    
    if response.status_code == 200:
        result = response.json()
        print("✅ Image captioning successful!")
        print(f"\n🖼️  Caption: {result['results']}")
        print(f"⏱️  Processing Time: {result['processing_time']:.4f}s")
    else:
        print(f"❌ Request failed: {response.text}")
        
except requests.exceptions.RequestException as e:
    print(f"❌ Request error: {e}")

## 4. Generation Test (With Reference)
Test text generation with product context

In [ ]:
# Test Generation with Reference
print("=== Testing Generation (With Reference) ===\n")

payload = {
    "question": "What sizes are available and how should I care for this item?",
    "reference": "Red summer dress, 100% cotton, available in multiple sizes, priced at $49.99, located on Floor 2 Section A"
}

try:
    response = requests.post(
        f"{BASE_URL}/generation",
        json=payload,
        headers=HEADERS,
        timeout=30
    )
    
    print(f"Status Code: {response.status_code}")
    print(f"Response Time: {response.elapsed.total_seconds():.2f}s")
    
    if response.status_code == 200:
        result = response.json()
        print("✅ Generation successful!")
        print(f"\n💬 Question: {payload['question']}")
        print(f"\n🤖 AI Response:\n{result['response']}")
    else:
        print(f"❌ Request failed: {response.text}")
        
except requests.exceptions.RequestException as e:
    print(f"❌ Request error: {e}")

In [ ]:
# Test Generation with Product Information Query
print("=== Testing Generation (Product Details) ===\n")

payload = {
    "question": "Can you tell me more about the materials and why this would be good for summer?",
    "reference": "Classic red summer dress perfect for casual occasions. Materials: 100% Cotton. Brand: The Mall Fashion. Price: $49.99. Location: Floor 2, Section A"
}

try:
    response = requests.post(
        f"{BASE_URL}/generation",
        json=payload,
        headers=HEADERS,
        timeout=30
    )
    
    print(f"Status Code: {response.status_code}")
    print(f"Response Time: {response.elapsed.total_seconds():.2f}s")
    
    if response.status_code == 200:
        result = response.json()
        print("✅ Generation successful!")
        print(f"\n💬 Question: {payload['question']}")
        print(f"\n🤖 AI Response:\n{result['response']}")
    else:
        print(f"❌ Request failed: {response.text}")
        
except requests.exceptions.RequestException as e:
    print(f"❌ Request error: {e}")

## 5. Generation Test (Without Reference)
Test what happens when no product context is provided

In [ ]:
# Test Generation without Reference
print("=== Testing Generation (No Reference) ===\n")

payload = {
    "question": "What products do you have available?",
    "reference": ""
}

try:
    response = requests.post(
        f"{BASE_URL}/generation",
        json=payload,
        headers=HEADERS,
        timeout=15
    )
    
    print(f"Status Code: {response.status_code}")
    print(f"Response Time: {response.elapsed.total_seconds():.2f}s")
    
    if response.status_code == 200:
        result = response.json()
        print("✅ Generation successful!")
        print(f"\n💬 Question: {payload['question']}")
        print(f"\n🤖 AI Response:\n{result['response']}")
    else:
        print(f"❌ Request failed: {response.text}")
        
except requests.exceptions.RequestException as e:
    print(f"❌ Request error: {e}")

## 6. End-to-End Workflow Test
Test a complete workflow: search → generate response

In [ ]:
# End-to-End Workflow Test
print("=== End-to-End Workflow Test ===\n")

# Step 1: Search for products
print("Step 1: Searching for leather shoes...")
search_payload = {
    "user_query": "I need leather shoes for business meetings",
    "image_prompt": "professional business footwear"
}

try:
    search_response = requests.post(
        f"{BASE_URL}/finding_documents",
        json=search_payload,
        headers=HEADERS,
        timeout=30
    )
    
    if search_response.status_code == 200:
        search_result = search_response.json()
        print(f"✅ Found {len(search_result['results'])} products")
        
        # Get the first product
        if search_result['results']:
            product = search_result['results'][0]
            print(f"\n📦 Selected Product: {product['brand_name']}")
            print(f"Description: {product['description']}")
            
            # Step 2: Generate response about the product
            print("\nStep 2: Asking AI about the product...")
            
            # Create reference from product data
            reference = f"{product['description']} - Brand: {product['brand_name']}, Price: {product['price']}, Materials: {product['materials']}, Location: {product['location']}"
            
            gen_payload = {
                "question": "Are these shoes suitable for daily business wear and what's special about them?",
                "reference": reference
            }
            
            gen_response = requests.post(
                f"{BASE_URL}/generation",
                json=gen_payload,
                headers=HEADERS,
                timeout=30
            )
            
            if gen_response.status_code == 200:
                gen_result = gen_response.json()
                print("✅ AI response generated successfully!")
                print(f"\n🤖 AI Recommendation:\n{gen_result['response']}")
            else:
                print(f"❌ Generation failed: {gen_response.text}")
        else:
            print("❌ No products found")
    else:
        print(f"❌ Search failed: {search_response.text}")
        
except requests.exceptions.RequestException as e:
    print(f"❌ Request error: {e}")

print("\n🎉 End-to-End workflow test completed!")

## 7. Performance Summary
Quick overview of all endpoint performance

In [ ]:
# Performance Test Summary
print("=== API Performance Summary ===\n")

endpoints = [
    {
        'name': 'Health Check',
        'method': 'GET',
        'url': f"{BASE_URL}/health",
        'payload': None
    },
    {
        'name': 'Finding Documents',
        'method': 'POST',
        'url': f"{BASE_URL}/finding_documents",
        'payload': {"user_query": "red dress", "image_prompt": "clothing"}
    },
    {
        'name': 'Image Captioning',
        'method': 'POST',
        'url': f"{BASE_URL}/image_captioning",
        'payload': {"image_path": "test.jpg"}
    },
    {
        'name': 'Generation (No Ref)',
        'method': 'POST',
        'url': f"{BASE_URL}/generation",
        'payload': {"question": "Hello", "reference": ""}
    }
]

performance_data = []

for endpoint in endpoints:
    try:
        if endpoint['method'] == 'GET':
            response = requests.get(endpoint['url'], timeout=10)
        else:
            response = requests.post(
                endpoint['url'],
                json=endpoint['payload'],
                headers=HEADERS,
                timeout=15
            )
        
        performance_data.append({
            'Endpoint': endpoint['name'],
            'Status': response.status_code,
            'Response Time (s)': f"{response.elapsed.total_seconds():.2f}",
            'Status Text': '✅ Success' if response.status_code == 200 else '❌ Failed'
        })
        
    except Exception as e:
        performance_data.append({
            'Endpoint': endpoint['name'],
            'Status': 'Error',
            'Response Time (s)': 'N/A',
            'Status Text': f'❌ {str(e)[:30]}...'
        })

# Display results as a table
df = pd.DataFrame(performance_data)
print(df.to_string(index=False))

print(f"\n⏰ Test completed at: {datetime.now()}")

## 8. Error Handling Tests
Test various error scenarios

In [ ]:
# Test Error Handling
print("=== Testing Error Handling ===\n")

# Test 1: Invalid JSON
print("Test 1: Invalid request data")
try:
    response = requests.post(
        f"{BASE_URL}/finding_documents",
        json={"invalid_field": "test"},  # Missing required fields
        headers=HEADERS,
        timeout=10
    )
    print(f"Status: {response.status_code} (Expected: 422 Validation Error)")
    if response.status_code == 422:
        print("✅ Validation error handled correctly")
    print(f"Response: {response.text[:100]}...\n")
except Exception as e:
    print(f"❌ Error: {e}\n")

# Test 2: Empty strings
print("Test 2: Empty string inputs")
try:
    response = requests.post(
        f"{BASE_URL}/finding_documents",
        json={"user_query": "", "image_prompt": ""},
        headers=HEADERS,
        timeout=10
    )
    print(f"Status: {response.status_code}")
    if response.status_code == 200:
        print("✅ Empty inputs handled gracefully")
    print(f"Response: {response.text[:100]}...\n")
except Exception as e:
    print(f"❌ Error: {e}\n")

# Test 3: Non-existent endpoint
print("Test 3: Non-existent endpoint")
try:
    response = requests.get(f"{BASE_URL}/nonexistent", timeout=5)
    print(f"Status: {response.status_code} (Expected: 404 Not Found)")
    if response.status_code == 404:
        print("✅ 404 error handled correctly")
except Exception as e:
    print(f"❌ Error: {e}")

print("\n🧪 Error handling tests completed!")